In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18_cutout_cifar100.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [3]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01
mean = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std = [0.2673342858792401, 0.2564384629170883, 0.27615047132568404]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='data', train=True, download=False, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR100(root='data', train=False, download=False, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=100):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [5]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

In [7]:
checkpoint = None
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location = 'cuda')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    # 作用cutout
    cut = Cutout(n_holes = 1, length = 16)
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        for dd in range(inputs.size(0)):
            if random.random()<0.5:
                inputs[dd] = cut(inputs[dd])
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18_cutout_cifar100.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18_cutout_cifar100.txt", "a")
    f2.write('resnet_cifar100_cutout测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('测试分类准确率为：%.3f%%' % (100 * correct / total))
        f3 = open("test_acc_resnet18_cutout_cifar100.txt", "a")
        f3.write('resnet_cifar100_cutout测试分类准确率为：%.3f%%\n' % (100 * correct / total))
        f3.close()
        acc = 100. * correct / total
        best_acc = 85
        if acc > best_acc:
            f3 = open("best_acc_resnet18_cutout_cifar100.txt", "w")
            f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
            f3.close()
            best_acc = acc
if __name__ == '__main__':
    main()

Epoch: [0][0/391]	Batch Time 2.831 (2.831)	Data Time 0.000 (0.000)	Loss 4.6788 (4.6788)	
Epoch: [0][20/391]	Batch Time 0.187 (0.307)	Data Time 0.000 (0.000)	Loss 4.4261 (4.5652)	
Epoch: [0][40/391]	Batch Time 0.187 (0.247)	Data Time 0.000 (0.000)	Loss 4.3576 (4.4586)	
Epoch: [0][60/391]	Batch Time 0.187 (0.225)	Data Time 0.000 (0.000)	Loss 4.1585 (4.3618)	
Epoch: [0][80/391]	Batch Time 0.187 (0.214)	Data Time 0.000 (0.000)	Loss 4.0605 (4.2875)	
Epoch: [0][100/391]	Batch Time 0.172 (0.208)	Data Time 0.000 (0.000)	Loss 4.0057 (4.2236)	
Epoch: [0][120/391]	Batch Time 0.187 (0.203)	Data Time 0.000 (0.000)	Loss 3.7874 (4.1671)	
Epoch: [0][140/391]	Batch Time 0.172 (0.200)	Data Time 0.000 (0.000)	Loss 3.9703 (4.1167)	
Epoch: [0][160/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 3.7972 (4.0786)	
Epoch: [0][180/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 3.6361 (4.0411)	
Epoch: [0][200/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 3.5571 (4.0071)	
Epoch

Epoch: [4][180/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 2.0876 (2.0713)	
Epoch: [4][200/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 1.8091 (2.0753)	
Epoch: [4][220/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 1.8817 (2.0725)	
Epoch: [4][240/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 1.9170 (2.0716)	
Epoch: [4][260/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 1.7596 (2.0614)	
Epoch: [4][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 1.9591 (2.0549)	
Epoch: [4][300/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 2.0822 (2.0526)	
Epoch: [4][320/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 2.1089 (2.0521)	
Epoch: [4][340/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 2.1701 (2.0469)	
Epoch: [4][360/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 1.9625 (2.0405)	
Epoch: [4][380/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Loss 1.9834 (2.0358)	

Epoch: [8][360/391]	Batch Time 0.192 (0.191)	Data Time 0.000 (0.000)	Loss 1.5538 (1.4960)	
Epoch: [8][380/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 1.5221 (1.4974)	
tensor(0.5770)
测试分类准确率为：50.840%
Epoch: [9][0/391]	Batch Time 2.768 (2.768)	Data Time 0.000 (0.000)	Loss 1.2325 (1.2325)	
Epoch: [9][20/391]	Batch Time 0.172 (0.306)	Data Time 0.000 (0.000)	Loss 1.2443 (1.4131)	
Epoch: [9][40/391]	Batch Time 0.187 (0.247)	Data Time 0.000 (0.000)	Loss 1.4207 (1.3853)	
Epoch: [9][60/391]	Batch Time 0.187 (0.226)	Data Time 0.000 (0.000)	Loss 1.4304 (1.3910)	
Epoch: [9][80/391]	Batch Time 0.187 (0.215)	Data Time 0.000 (0.000)	Loss 1.5365 (1.3892)	
Epoch: [9][100/391]	Batch Time 0.187 (0.209)	Data Time 0.000 (0.000)	Loss 1.5178 (1.3900)	
Epoch: [9][120/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 1.4753 (1.3946)	
Epoch: [9][140/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 1.3531 (1.3944)	
Epoch: [9][160/391]	Batch Time 0.187 (0.200)	Data Time 0.000 (0.

Epoch: [13][120/391]	Batch Time 0.172 (0.205)	Data Time 0.000 (0.000)	Loss 1.2708 (1.1426)	
Epoch: [13][140/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 1.0065 (1.1372)	
Epoch: [13][160/391]	Batch Time 0.187 (0.200)	Data Time 0.000 (0.000)	Loss 1.2420 (1.1345)	
Epoch: [13][180/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 1.2986 (1.1391)	
Epoch: [13][200/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 1.0870 (1.1399)	
Epoch: [13][220/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 1.1062 (1.1433)	
Epoch: [13][240/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 1.4065 (1.1490)	
Epoch: [13][260/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 1.0905 (1.1497)	
Epoch: [13][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 1.1477 (1.1506)	
Epoch: [13][300/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 1.1966 (1.1529)	
Epoch: [13][320/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 1.334

Epoch: [17][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.9492 (0.9629)	
Epoch: [17][300/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.9296 (0.9679)	
Epoch: [17][320/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.9463 (0.9692)	
Epoch: [17][340/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 1.1437 (0.9697)	
Epoch: [17][360/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.9242 (0.9707)	
Epoch: [17][380/391]	Batch Time 0.171 (0.191)	Data Time 0.000 (0.000)	Loss 0.9912 (0.9722)	
tensor(0.7099)
测试分类准确率为：61.560%
Epoch: [18][0/391]	Batch Time 2.954 (2.954)	Data Time 0.000 (0.000)	Loss 0.9389 (0.9389)	
Epoch: [18][20/391]	Batch Time 0.183 (0.315)	Data Time 0.000 (0.000)	Loss 0.9239 (0.9125)	
Epoch: [18][40/391]	Batch Time 0.183 (0.252)	Data Time 0.000 (0.000)	Loss 0.7941 (0.9397)	
Epoch: [18][60/391]	Batch Time 0.183 (0.230)	Data Time 0.000 (0.000)	Loss 0.9119 (0.9232)	
Epoch: [18][80/391]	Batch Time 0.183 (0.219)	Data Tim

...

Epoch: [179][260/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.0360 (0.0426)	
Epoch: [179][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.0245 (0.0422)	
Epoch: [179][300/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0163 (0.0425)	
Epoch: [179][320/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 0.0324 (0.0421)	
Epoch: [179][340/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0410 (0.0416)	
Epoch: [179][360/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0427 (0.0417)	
Epoch: [179][380/391]	Batch Time 0.187 (0.190)	Data Time 0.000 (0.000)	Loss 0.0273 (0.0416)	
tensor(0.9905)
测试分类准确率为：76.380%
Epoch: [180][0/391]	Batch Time 2.739 (2.739)	Data Time 0.000 (0.000)	Loss 0.0241 (0.0241)	
Epoch: [180][20/391]	Batch Time 0.187 (0.305)	Data Time 0.000 (0.000)	Loss 0.0230 (0.0352)	
Epoch: [180][40/391]	Batch Time 0.172 (0.246)	Data Time 0.000 (0.000)	Loss 0.0712 (0.0358)	
Epoch: [180][60/391]	Batch Time 0.187 (0.2

tensor(0.9916)
测试分类准确率为：76.480%
Epoch: [184][0/391]	Batch Time 2.755 (2.755)	Data Time 0.000 (0.000)	Loss 0.0127 (0.0127)	
Epoch: [184][20/391]	Batch Time 0.172 (0.306)	Data Time 0.000 (0.000)	Loss 0.0331 (0.0347)	
Epoch: [184][40/391]	Batch Time 0.187 (0.247)	Data Time 0.000 (0.000)	Loss 0.0339 (0.0343)	
Epoch: [184][60/391]	Batch Time 0.172 (0.226)	Data Time 0.000 (0.000)	Loss 0.0297 (0.0353)	
Epoch: [184][80/391]	Batch Time 0.187 (0.216)	Data Time 0.000 (0.000)	Loss 0.0300 (0.0365)	
Epoch: [184][100/391]	Batch Time 0.187 (0.209)	Data Time 0.000 (0.000)	Loss 0.0493 (0.0348)	
Epoch: [184][120/391]	Batch Time 0.172 (0.205)	Data Time 0.000 (0.000)	Loss 0.0362 (0.0362)	
Epoch: [184][140/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 0.0355 (0.0373)	
Epoch: [184][160/391]	Batch Time 0.187 (0.200)	Data Time 0.000 (0.000)	Loss 0.0451 (0.0378)	
Epoch: [184][180/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 0.0810 (0.0388)	
Epoch: [184][200/391]	Batch Time 0.187 (0.19

Epoch: [188][140/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 0.0159 (0.0403)	
Epoch: [188][160/391]	Batch Time 0.187 (0.199)	Data Time 0.000 (0.000)	Loss 0.0433 (0.0405)	
Epoch: [188][180/391]	Batch Time 0.187 (0.198)	Data Time 0.000 (0.000)	Loss 0.0519 (0.0401)	
Epoch: [188][200/391]	Batch Time 0.187 (0.196)	Data Time 0.000 (0.000)	Loss 0.0279 (0.0400)	
Epoch: [188][220/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Loss 0.0427 (0.0397)	
Epoch: [188][240/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 0.0216 (0.0397)	
Epoch: [188][260/391]	Batch Time 0.187 (0.194)	Data Time 0.000 (0.000)	Loss 0.0516 (0.0397)	
Epoch: [188][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.0379 (0.0404)	
Epoch: [188][300/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0333 (0.0406)	
Epoch: [188][320/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0185 (0.0407)	
Epoch: [188][340/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)

Epoch: [192][280/391]	Batch Time 0.187 (0.193)	Data Time 0.000 (0.000)	Loss 0.0503 (0.0397)	
Epoch: [192][300/391]	Batch Time 0.172 (0.192)	Data Time 0.000 (0.000)	Loss 0.0999 (0.0401)	
Epoch: [192][320/391]	Batch Time 0.187 (0.192)	Data Time 0.000 (0.000)	Loss 0.0663 (0.0402)	
Epoch: [192][340/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0262 (0.0406)	
Epoch: [192][360/391]	Batch Time 0.187 (0.191)	Data Time 0.000 (0.000)	Loss 0.0620 (0.0407)	
Epoch: [192][380/391]	Batch Time 0.172 (0.191)	Data Time 0.000 (0.000)	Loss 0.0423 (0.0406)	
tensor(0.9913)
测试分类准确率为：76.370%
Epoch: [193][0/391]	Batch Time 2.756 (2.756)	Data Time 0.000 (0.000)	Loss 0.0371 (0.0371)	
Epoch: [193][20/391]	Batch Time 0.187 (0.306)	Data Time 0.000 (0.000)	Loss 0.0170 (0.0366)	
Epoch: [193][40/391]	Batch Time 0.187 (0.247)	Data Time 0.000 (0.000)	Loss 0.0245 (0.0378)	
Epoch: [193][60/391]	Batch Time 0.195 (0.226)	Data Time 0.000 (0.000)	Loss 0.0373 (0.0379)	
Epoch: [193][80/391]	Batch Time 0.172 (0.21

Epoch: [197][20/391]	Batch Time 0.172 (0.305)	Data Time 0.000 (0.000)	Loss 0.0213 (0.0345)	
Epoch: [197][40/391]	Batch Time 0.172 (0.246)	Data Time 0.000 (0.000)	Loss 0.0199 (0.0377)	
Epoch: [197][60/391]	Batch Time 0.187 (0.226)	Data Time 0.000 (0.000)	Loss 0.0993 (0.0412)	
Epoch: [197][80/391]	Batch Time 0.187 (0.215)	Data Time 0.000 (0.000)	Loss 0.0238 (0.0413)	
Epoch: [197][100/391]	Batch Time 0.181 (0.209)	Data Time 0.000 (0.000)	Loss 0.0482 (0.0414)	
Epoch: [197][120/391]	Batch Time 0.187 (0.205)	Data Time 0.000 (0.000)	Loss 0.0271 (0.0403)	
Epoch: [197][140/391]	Batch Time 0.187 (0.202)	Data Time 0.000 (0.000)	Loss 0.0395 (0.0412)	
Epoch: [197][160/391]	Batch Time 0.187 (0.200)	Data Time 0.000 (0.000)	Loss 0.0265 (0.0411)	
Epoch: [197][180/391]	Batch Time 0.172 (0.198)	Data Time 0.000 (0.000)	Loss 0.0316 (0.0409)	
Epoch: [197][200/391]	Batch Time 0.172 (0.196)	Data Time 0.000 (0.000)	Loss 0.0203 (0.0414)	
Epoch: [197][220/391]	Batch Time 0.187 (0.195)	Data Time 0.000 (0.000)	Los